# Mushroom data set

The Neural network was run on the mushroom data set and the goal was to classify the mushroom as poisonous or edible. The data set has 22 input values and since all of the were catogorical variable, it was converted to numerical variable using the get_dummies method in the pandas library, this converted the data set into a data set with 118 columns. 

It took a lot of time of time to figure out what initial weights to input, what activation functions to use, how to traing the data( whether in batches or not) etc. After some experimenting I decided to initialize the weights with a normal distribution with mean 0 and standard deviation that depends on the dimension of the data. 

In this Neural network, I used the relu activation function for the hidden layers and the softmax function for the output layer. The model was trained with a batch size of 20, and then updating the weights (This was giving good results).  

In [29]:
# loading all the required libraries
import pandas as pd
import numpy as np
import math 
import copy
import pickle
import time

In [30]:
#preparing data, The data is split into 7000 train samples and, 1124 test samples
data = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/mushroom/agaricus-lepiota.data", header = None)
data.drop([12], axis = 1) # droping the column with missing values. 
data = pd.get_dummies(data, columns = data.columns)
datainput = data.iloc[:,2:]
dataoutput = data.iloc[:,[0,1]]
traininput = datainput.iloc[:7000,:].values
trainoutput = dataoutput.iloc[:7000,:].values
testinput = datainput.iloc[7000:,:].values
testoutput = dataoutput.iloc[7000:,:].values

In [31]:
# These are all the activation functions that I have trialed and errored with to make the model run.
def relu(x):
    if x > 0:
        return x
    else:
        return 0
 
def drelu(x):
    if x > 0:
        return 1
    else:
        return 0
 
def sigmoid(x):
    return 1/(1+math.exp(-x))
 
def dsigmoid(x):
    return sigmoid(x)*(1-sigmoid(x))
 
def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()
 
def dsoftmax(x):
    return (x * np.identity(x.size)- x.transpose() @ x)


In [32]:
# I started by trying to modularize everything, but ended up writing a single function that, trains and tests
# the loaded data. 
 
# Parameters:
# datainput - the training data
# dataoutput - training data output
# testinput - test data input
# testoutput - test data output
# layers - the number of layes in the network, this includes the number of hidden layers plus output layer.
# node_per_layer - the number of nodes in a layer,
# output - dimension of output/ number of classes. 
 
 
def NeuralNet(datainput, dataoutput, testinput, testoutput, layers, node_per_layer, output):

    begin = time.time() # this is to calculate the runtime. 
    # Setting up all initial weights and biases, I have set them up as dictionaries
    weights = {}
    bias = {}
    dweights = {}
    dbias = {}
    # These are set of dummy variables which I want in the shape of the weights, and their derivatives.
    cweights = {}
    cbias = {}
    rweights = {}
    rbias = {}
    dim = len(datainput[0])
    # Setting up initial weights and biases
    for i in range(layers):
        if i == 0:
            weights[i] = np.random.normal(0,(2/dim)**(0.5),size = (node_per_layer,dim))
            bias[i] = np.random.normal(0,1, size = (node_per_layer))
            dweights[i] = np.zeros((node_per_layer,dim))
            dbias[i] = np.zeros((node_per_layer))
            cweights[i] = np.zeros((node_per_layer,dim))
            cbias[i] = np.zeros((node_per_layer))
            rweights[i] = np.zeros((node_per_layer,dim))
            rbias[i] = np.zeros((node_per_layer))
        elif i == layers-1:
            weights[i] = np.random.uniform(-1,1,size = (output,node_per_layer))
            bias[i] = np.random.uniform(-1/2,1/2,size = (output))
            dweights[i] = np.random.uniform(1,5,size = (output,node_per_layer))
            dbias[i] = np.random.uniform(1,4,size = output)
            cweights[i] = np.zeros((output,node_per_layer))
            cbias[i] = np.zeros((output))
            rweights[i] = np.zeros((output,node_per_layer))
            rbias[i] = np.zeros((output))
        else:
            weights[i] = np.random.uniform(-1,1,size = (node_per_layer,node_per_layer))
            bias[i] = np.random.uniform(-1/2,1/2, size = (node_per_layer))
            dweights[i] = np.random.uniform(1,5,size = (node_per_layer,node_per_layer))
            dbias[i] = np.random.uniform(1,4,size = node_per_layer)
            cweights[i] = np.zeros((node_per_layer,node_per_layer))
            cbias[i] = np.zeros((node_per_layer))
            rweights[i] = np.zeros((node_per_layer,node_per_layer))
            rbias[i] = np.zeros((node_per_layer))
    
 
    a = {} #activation
    z = {} #dummy to find activation using relu and softmax output.
    delta = {} # catches the derivative of cost with respect to the dummy activation variables.
    for i in range(layers):
        if i == layers-1:
            a[i] = np.empty(output)
            z[i] = np.empty(output)
            delta[i] = np.empty(output)
        else:
            a[i] = np.empty(node_per_layer)
            z[i] = np.empty(node_per_layer)
            delta[i] = np.empty(node_per_layer)
 
    count = 0
    # Iteration through the whole data set.
    while count < 30:
        flag = 0
        itemcount = 0
        for item in range(len(datainput)): # these are inputs from the data set.
 
            # Calculating forwardpass with weights!
            for i in range(layers):
                if i == 0:
                    for j in range(node_per_layer):
                        z[i][j] = np.dot(datainput[item],weights[i][j]) + bias[i][j]
                        a[i][j] = relu(z[i][j])
                elif i == layers-1:
                    for j in range(output):
                        z[i][j] = np.dot(a[i-1],weights[i][j]) + bias[i][j]        
                    a[i] = softmax(z[i])
                else: 
                    for j in range(node_per_layer):
                        z[i][j] = np.dot(a[i-1],weights[i][j]) + bias[i][j]
                        a[i][j] = relu(z[i][j])
 
            value = a[layers-1] #this is the forwardpass output.
            dcost = np.empty((output)) 
            dvalue = dsoftmax(value) # the softmax deriative output
 
 
            # Backpropagation! 
 
            # We use mean square error as our cost function. 
            # try to capture the derivatives in the same dictionary form as the weights, biases and the dummy z variables. 
            # Derivative of cost with respect to the forward pass output.
            for i in range(output):
                dcost[i] = -(dataoutput[item][i] - value[i])
 
            # Finding the last layer gradients, all the gradient before this can be found from this. 
            for i in range(output):
                delta[layers-1][i] = np.dot(dcost,dvalue[:,i]) # derivative with respect to the dummy z -variable
                #delta[layers-1][i] = -2*(dataoutput[item][i]-value[i])*dsigmoid(z[layers-1][i])
                dbias[layers-1][i] =  delta[layers-1][i]
                for j in range(node_per_layer):
                    dweights[layers-1][i][j] = delta[layers-1][i]*a[layers-2][j]
 
            # Finding the gradient in the rest of the layers
            for i in range(layers-1):
                if i == layers-2:
                    for j in range(node_per_layer):
                        delta[layers-2-i][j] = np.dot(delta[layers-1-i],weights[layers-1-i][:,j])*drelu(z[layers-2-i][j])
                        dbias[layers-2-i][j] = delta[layers-2-i][j]
                        for k in range(dim):
                            dweights[layers-2-i][j][k] = delta[layers-2-i][j] * datainput[item][k]
                else:
                    for j in range(node_per_layer):
                        delta[layers-2-i][j] = np.dot(delta[layers-1-i],weights[layers-1-i][:,j])*drelu(z[layers-2-i][j])
                        dbias[layers-2-i][j] = delta[layers-2-i][j]
                        for k in range(node_per_layer):
                            dweights[layers-2-i][j][k] = delta[layers-2-i][j]*a[layers-3-i][k]
 
 
            # We train the model with batch size 20. i.e. find average gradient for 20 inputs and update the weights and biases
            for i in range(layers):
                cweights[i] = cweights[i] + dweights[i]
                cbias[i] = cbias[i] + dbias[i]
            
            # updating weights in batches of size 20 - This was found to give meainingful results as compared to updating weights after and epoch.
            # the 0.05 is the learning rate.
            if flag > 0 and flag % 20 == 0:
                for i in range(layers):
                    weights[i] = weights[i] -  0.01 * cweights[i]/20 # the weights are being updated
                    bias[i] = bias[i] -  0.01 * cbias[i]/20
                    cweights[i] = rweights[i]
                    cbias[i] = rbias[i]    
            
            flag += 1 
 
            #if itemcount < 6:
            #   print(value)
        
 
        count += 1 # counts epochs
        if count % 5 == 0:
            print("Iteration ", count, "is now complete")
    end1 = time.time()
    parameters = (weights, bias) # trained model weight!
    pickle.dump(parameters, open('sample_data/parameters1.p', 'wb')) # dumping the trained model weights and biases
    print("Training is now complete. It took", round((end1-begin)/60,2) ,"minutes to train the Network.")
 
    # Testing the model on untrained data.
    # Now we test for accuracy.
    correct = 0
    outcounter = 0
    for item in range(len(testinput)):
        outcounter +=1
 
        a = {} #activation
        z = {} #dummy to find activation using relu.
        for i in range(layers):
            if i == layers-1:
                a[i] = np.empty(output)
                z[i] = np.empty(output)
            else:
                a[i] = np.empty(node_per_layer)
                z[i] = np.empty(node_per_layer)
 
        # Calculating forwardpass with weights!
        for i in range(layers):
            if i == 0:
                for j in range(node_per_layer):
                    z[i][j] = np.dot(testinput[item],weights[i][j]) + bias[i][j]
                    a[i][j] = relu(z[i][j])
            elif i == layers-1:
                for j in range(output):
                    z[i][j] = np.dot(a[i-1],weights[i][j]) + bias[i][j]           
                a[i] = softmax(z[i])
            else: 
                for j in range(node_per_layer):
                    z[i][j] = np.dot(a[i-1],weights[i][j]) + bias[i][j]
                    a[i][j] = relu(z[i][j])
        value = a[layers-1]
        # if testoutput[item].index(np.amax(testoutput[item])) == value.index(np.amax(value)):
        #    correct += 1
        if np.linalg.norm(testoutput[item]-value) < 0.02: # testing accuracy!
            correct += 1
 
        # printing the last 5 model output and actual output!
        if outcounter == 1119:
            print('Printing last 5 Network output and test output:')
        if outcounter > 1119:
            print(value,testoutput[item])
    print("Testing is now complete.")
    print("Accuracy of the model is :", round(100 * correct/len(testinput),3)  ,"%")

# Results

Here we have used 3 hidden layers and 20 nodes per layer.

In [33]:
NeuralNet(traininput,trainoutput,testinput,testoutput,4,20,2)

Iteration  5 is now complete
Iteration  10 is now complete
Iteration  15 is now complete
Iteration  20 is now complete
Iteration  25 is now complete
Iteration  30 is now complete
Training is now complete. It took 15.26 minutes to train the Network.
Printing last 5 Network output and test output:
[9.99999976e-01 2.41486759e-08] [1 0]
[9.99999986e-01 1.38931807e-08] [1 0]
[9.99999978e-01 2.23768093e-08] [1 0]
[1.84999849e-04 9.99815000e-01] [0 1]
[9.99996447e-01 3.55341939e-06] [1 0]
Testing is now complete.
Accuracy of the model is : 97.242 %
